In [2]:
# import standard libraries
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import itertools

from sklearn.datasets import load_iris
from sklearn import tree, preprocessing
from sklearn.metrics import classification_report, confusion_matrix, average_precision_score, precision_recall_fscore_support,f1_score
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from shutil import copyfile


Bad key "text.kerning_factor" on line 4 in
/Users/mitch/opt/anaconda3/envs/ml/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
# load train data
train_data = pd.read_csv('../data/MoreManipulatedData_bucket_last_col.csv',usecols=lambda x: 'PLAYER_URL' not in x)

# different classes
train_data.BUCKET.unique()

# convert string values to numerical data
def convert(data):
    number = preprocessing.LabelEncoder()
    data['POS'] = number.fit_transform(data.POS)
    data['LEAGUE'] = number.fit_transform(data.LEAGUE)
    data['FIRST_JUNIOR_YEAR'] = number.fit_transform(data.FIRST_JUNIOR_YEAR)
    data['DOB'] = number.fit_transform(data.DOB)
    data['NATIONALITY'] = number.fit_transform(data.NATIONALITY)
    data['SHOOTS'] = number.fit_transform(data.SHOOTS)
    data=data.fillna(-999)
    return data

# convert string values in data to numerical classes
train_data = convert(train_data)

# Separating the data and the labels
X = np.asarray(train_data[train_data.columns[:-1]])
y = np.asarray(train_data.BUCKET)

# Splitting the data into the train and the test sets
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
sss.get_n_splits(X, y)

train_index, test_index = next(sss.split(X, y))
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]

print('Training data: \n',X)
print('\n')
print('Training labels: \n',y_train)

Training data: 
 [[103143      3     87 ...    180     84      2]
 [106210      1     73 ...    193     95      0]
 [106207      2     21 ...    183     86      0]
 ...
 [249307      1    130 ...    188     98      3]
 [485033      3     59 ...    175     68      2]
 [410552      2      1 ...    183     82      2]]


Training labels: 
 ['0' '0' '0' ... '0' '0' '0']


In [4]:
# K Nearest Neighbors Classifier (has to be able to deal with floats)
MLP = MLPClassifier(solver= 'adam', learning_rate_init = 0.000001, max_iter=10000,random_state=0)

# learning_rate_init=0.0001,learning_rate='invscaling',activation='logistic'
# fit the classifier using the training data
MLP = MLP.fit(X_train, y_train)

# Predict the test class labels using the trained KNN classifier 
y_pred = MLP.predict(X_test)

# print accuracy of the classifier
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    1-200          0.28      0.17      0.21       408
    201-600        0.14      0.05      0.08       156
    601+           0.34      0.35      0.35       127
           0       0.93      0.97      0.95      6009

    accuracy                           0.89      6700
   macro avg       0.42      0.39      0.40      6700
weighted avg       0.86      0.89      0.87      6700



In [9]:
print(MLP.n_layers_)
print(precision_recall_fscore_support(y_test, y_pred)[2][3])
print(f1_score(y_test, y_pred,average='macro'))
print(f1_score(y_test, y_pred,average='micro'))
print(f1_score(y_test, y_pred,average='weighted'))

3
0.9489288914229861
0.3951114950012613
0.8874626865671642
0.8721191515809892


In [70]:
from joblib import dump, load
dump(MLP, '../finished_models/mlp_adam_lr0_00001.joblib') 

['../finished_models/mlp_adam_lr0_00001.joblib']

In [71]:
# This is a champ: '../finished_models/mlp_hid1000_layer3_try2.joblib'
MLP2 = load('../finished_models/mlp_adam_lr0_00001.joblib') 
# Predict the test class labels using the trained KNN classifier 
y_pred = MLP2.predict(X_test)

# print accuracy of the classifier
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    1-200          0.23      0.37      0.28       408
    201-600        0.21      0.07      0.11       156
    601+           0.35      0.35      0.35       127
           0       0.95      0.92      0.93      6009

    accuracy                           0.86      6700
   macro avg       0.43      0.43      0.42      6700
weighted avg       0.87      0.86      0.86      6700

